# GPU-Accelerated Quantitative Portfolio Optimization with NVIDIA cuOpt™
[![Click here to deploy.](https://brev-assets.s3.us-west-1.amazonaws.com/nv-lb-dark.svg)](https://brev.nvidia.com/launchable/deploy?launchableID=env-360InRZzyHqDnJYQKIxaSggF8xI)

This launchable introduces NVIDIA's GPU-Accelerated Portfolio Optimization developer blueprint. We use the Mean-CVaR model to solve the problem of portfolio asset allocation. We show step-by-step how to pre-process input data, build the optimization model, solve the problem, and backtest the portfolio performance.

In particular, we will cover the following: 
- **Portfolio Optimization Setup** – Clone repository, install dependencies, and configure environment
- **Data-preprocessing** - Calculate daily returns from input price data 
- **Scenario generation** - Use cuML KDE to model returns distribution and generate return scenarios 
- **Model Building** - Build the optimization problem with optional constraints
- **Solving Optimization on GPU** – Call GPU/CPU solvers with customizable settings to solve the problem
- **Backtesting** - Visualize and evaluate optimized portfolio
- **Comparing CPU and GPU Performance** – Evaluate performance solving optimization problem on GPU vs CPU
- **Generating Efficient Frontier** - Create an efficient frontier by solving optimization for different risk aversion levels
- **Backtesting Re-Balancing Strategies** - Re-balance portfolios over time with different strategies

In this setup notebook, we first focus on the essential steps required to prepare the environment and install all necessary dependencies for the Quantitative Portfolio Optimization example. Once the setup is complete, you need to proceed to the main notebook, [launchable-content.ipynb](/quantitative-portfolio-optimization/brev/launchable-content.ipynb).

---

<a id="environment-setup"></a>
## Environment Setup

Run the cells below to clone the repo, configure project path, install dependencies, and configure the Jupyter kernel.

**Note**: only clone the GitHub repository when deploying the Brev launchable! 


In [1]:
!git clone https://github.com/NVIDIA-AI-Blueprints/quantitative-portfolio-optimization.git

Cloning into 'quantitative-portfolio-optimization'...
remote: Enumerating objects: 473, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 473 (delta 36), reused 37 (delta 25), pack-reused 412 (from 1)
Receiving objects: 100% (473/473), 18.35 MiB | 18.83 MiB/s, done.
Resolving deltas: 100% (257/257), done.


In [2]:
from pathlib import Path
import os
notebook_dir = Path.cwd()
# For Brev Users
project_root = notebook_dir / 'quantitative-portfolio-optimization'

local_bin = os.path.expanduser("~/.local/bin")
os.environ["PATH"] = f"{local_bin}:{os.environ['PATH']}"
print(f"Updated PATH to include: {local_bin}")

import sys

data_dir = project_root / "data"
sys.path.insert(0, str(project_root))
os.chdir(project_root)
print(f"Working directory changed to: {Path.cwd()}")

Updated PATH to include: /home/nvidia/.local/bin
Working directory changed to: /home/nvidia/quantitative-portfolio-optimization


### Install Dependencies


In [3]:
import subprocess
import re
import os

def detect_cuda_version():
    """Detect CUDA runtime version from nvidia-smi or nvcc."""
    # Try nvidia-smi first (gets driver's CUDA version)
    try:
        result = subprocess.run(["nvidia-smi"], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            match = re.search(r"CUDA Version:\s*(\d+)\.(\d+)", result.stdout)
            if match:
                major = int(match.group(1))
                print(f"✓ Detected CUDA {major}.{match.group(2)} (from nvidia-smi)")
                return major
    except Exception:
        pass
    
    # Fallback to nvcc
    try:
        result = subprocess.run(["nvcc", "--version"], capture_output=True, text=True, timeout=5)
        if result.returncode == 0:
            match = re.search(r"release (\d+)\.(\d+)", result.stdout)
            if match:
                major = int(match.group(1))
                print(f"✓ Detected CUDA {major}.{match.group(2)} (from nvcc)")
                return major
    except Exception:
        pass
    
    print("⚠ CUDA not detected, defaulting to cu12")
    return 12

CUDA_MAJOR = detect_cuda_version()
CUDA_EXTRA = "cuda12" if CUDA_MAJOR <= 12 else "cuda13"
print(f"→ Will install with extra: {CUDA_EXTRA}")

# Export for use in install cell
os.environ["CUDA_SUFFIX"] = CUDA_EXTRA


✓ Detected CUDA 12.4 (from nvidia-smi)
→ Will install with extra: cuda12


### Install Dependencies


In [4]:
import subprocess

# Get CUDA suffix from environment (set by detection cell)
cuda_suffix = os.environ.get("CUDA_SUFFIX", "cu12")
print(f"Installing with CUDA extra: {cuda_suffix}")

# Install uv and sync with appropriate CUDA extra
subprocess.run("curl -sSf https://astral.sh/uv/install.sh | sh", shell=True, check=True)

install_script = f"""
unset VIRTUAL_ENV
uv sync --extra {cuda_suffix}
uv run python -m ipykernel install --user --name=portfolio-opt --display-name "Portfolio Optimization"
"""

subprocess.run(install_script, shell=True, executable="/bin/bash")
print(f"\n✅ Installed with --extra {cuda_suffix}")

Installing with CUDA extra: cuda12


Using CPython 3.12.12
Creating virtual environment at: .venv
Resolved 201 packages in 0.68ms
Prepared 138 packages in 57.69s
Installed 138 packages in 168ms
 + asttokens==3.0.1
 + beautifulsoup4==4.14.2
 + cachetools==6.2.2
 + certifi==2025.11.12
 + cffi==2.0.0
 + charset-normalizer==3.4.4
 + clarabel==0.11.1
 + click==8.3.1
 + cloudpickle==3.1.2
 + comm==0.2.3
 + contourpy==1.3.3
 + cuda-bindings==12.9.4
 + cuda-core==0.3.2
 + cuda-pathfinder==1.3.2
 + cuda-python==12.9.4
 + cuda-toolkit==12.9.1
 + cudf-cu12==25.10.0
 + cufolio==25.10 (from file:///home/nvidia/quantitative-portfolio-optimization)
 + cuml-cu12==25.10.0
 + cuopt-cu12==25.10.1
 + cuopt-mps-parser==25.10.1
 + cupy-cuda12x==13.6.0
 + curl-cffi==0.13.0
 + cuvs-cu12==25.10.0
 + cvxpy==1.7.3
 + cycler==0.12.1
 + dask==2025.9.1
 + dask-cuda==25.10.0
 + dask-cudf-cu12==25.10.0
 + debugpy==1.8.17
 + decorator==5.2.1
 + distributed==2025.9.1
 + distributed-ucxx-cu12==0.46.0
 + executing==2.2.1
 + fastrlock==0.8.3
 + fonttools==4.

Installed kernelspec portfolio-opt in /home/nvidia/.local/share/jupyter/kernels/portfolio-opt

✅ Installed with --extra cuda12


### Verify Installation

After running the cell above, **select the *Portfolio Optimization* kernel** from the kernel menu, then run the cell below to verify all packages are installed.

In [1]:
import importlib

packages = ['numpy', 'pandas', 'cvxpy', 'sklearn', 'seaborn', 'cuml', 'cuopt', 'yfinance']

print("Checking packages...\n")
failed = []
for pkg in packages:
    try:
        mod = importlib.import_module(pkg)
        ver = getattr(mod, '__version__', 'installed')
        print(f"✓ {pkg:12s} {ver}")
    except ImportError as e:
        print(f"✗ {pkg:12s} FAILED")
        failed.append(pkg)

print(f"\n{'❌ Missing: ' + ', '.join(failed) if failed else '✅ All packages ready!'}")

Checking packages...

✓ numpy        2.2.6
✓ pandas       2.3.2
✓ cvxpy        1.7.3
✓ sklearn      1.7.1
✓ seaborn      0.13.2
✓ cuml         25.10.00
✓ cuopt        25.10.01
✓ yfinance     0.2.66

✅ All packages ready!


---
## Setup Complete! 🎉

If you see the message **✅ All packages ready!** above, your Python environment and kernel have been successfully configured.

You can now proceed to the main quantitative analysis notebook:

➡️ **[Click here to start the Portfolio Optimization Notebook](./quantitative-portfolio-optimization/brev/launchable-content.ipynb)** ⬅️

---

SPDX-FileCopyrightText: Copyright (c) 2023-2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved. 

SPDX-License-Identifier: Apache-2.0

Licensed under the Apache License, Version 2.0 (the "License");you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.